In [14]:
import cadquery as cq
# from lib import *
from random import *
from math import *
from perlin_noise import PerlinNoise

width = 90
height = 45

size = 1.5
min_extr = 3.0
max_extr = 12.0


noise = PerlinNoise(octaves=2, seed=1)


def sin_norm(phase):
    return sin(phase) * 0.5 + 0.5


def dist(v1: cq.Vector, v2: cq.Vector):
    return sqrt((v1.x - v2.x) ** 2 + (v1.y - v2.y) ** 2 + (v1.z - v2.z) ** 2)


def angle(v1: cq.Vector, v2: cq.Vector):
    return atan2(v1.y - v2.y, v1.x - v2.x)


def extrude_wave(loc: cq.Vector):
    center = cq.Vector(0.0, 0.0, 0.0)
    d = dist(loc, center)
    a = angle(loc, center)
    extrude_depth = sin_norm(pow(d * 0.07, 4.0) + a * 2)
    extrude_depth = extrude_depth * (max_extr - min_extr) + min_extr

    return cq.Workplane("XY", loc).rect(size, size).extrude(extrude_depth).val()


def extrude_perlin(loc: cq.Vector):
    perlin_factor = 0.03
    p = noise((abs(loc.x) * perlin_factor, loc.y * perlin_factor)) + 0.5
    extrude_depth = p * (max_extr - min_extr) + min_extr

    return cq.Workplane("XY", loc).rect(size, size).extrude(extrude_depth).val()


sound_absorber = (
    cq.Workplane("XY")
    .rarray(size, size, round(width / size), round(height / size))
    .each(lambda vec: extrude_perlin(vec))
    # .each(lambda vec: extrude_wave(vec))
)


print("exporting...")
cq.exporters.export(sound_absorber, "sound_absorber.stl")
print("exported")

exporting...
exported


In [41]:
import cadquery as cq
import os

#Create a box geometry
length = 1.5
width = 1.0
height = 0.8
box = cq.Workplane("XY").box(length, width, height)
step_filename = "box.step"
path = os.path.join("results", step_filename)
cq.exporters.export(box, path)


*******************************************************************
******        Statistics on Transfer (Write)                 ******

*******************************************************************
******        Transfer Mode = 0  I.E.  As Is       ******
******        Transferring Shape, ShapeType = 0                      ******
** WorkSession : Sending all data
 Step File Name : results/box.step(350 ents)  Write  Done


In [36]:
import gmsh
gmsh.initialize()

In [26]:
import gmsh

# Initialize Gmsh
gmsh.initialize()

# Create a new model
gmsh.model.add("MyModel")

# Load the STL file
gmsh.merge(path)

# Synchronize the model
gmsh.model.mesh.generate(3)

# Get the mesh entities
mesh = gmsh.model.mesh.getEntities()

# Add physical groups to the mesh entities
gmsh.model.addPhysicalGroup(2, [ent[1] for ent in mesh if ent[0] == 2], label=1)  # Surface physical group
gmsh.model.addPhysicalGroup(3, [ent[1] for ent in mesh if ent[0] == 3], label=1)  # Volume physical group

# Define a volume from the surface entities
gmsh.model.addVolume([1], tag=1)

# Save the mesh
gmsh.write("test_mesh.msh")

# Finalize Gmsh
gmsh.finalize()


Info    : Reading 'results/box.stl'...
Info    : Mesh is in binary format
Info    : 12 facets in solid 0 STL Exported by Open CASCADE Technology [dev.opencascade.org]
Info    : Done reading 'results/box.stl'
Info    : Meshing 1D...
Info    : Done meshing 1D (Wall 8.335e-06s, CPU 6.8e-05s)
Info    : Meshing 2D...
Info    : Done meshing 2D (Wall 2.0864e-05s, CPU 1.3e-05s)
Info    : Meshing 3D...
Info    : Done meshing 3D (Wall 1.2646e-05s, CPU 8e-06s)
Info    : 8 nodes 12 elements


AttributeError: type object 'mesh' has no attribute 'getEntities'

In [34]:
import dolfinx
from dolfinx.io import gmshio
import gmsh
from mpi4py import MPI

# Load the mesh using Dolfinx
# mesh, cell_tags, facet_tags = gmshio.read_from_mesh(path, comm=MPI.COMM_WORLD)
mesh, cell_tags, facet_tags = gmshio.read_from_msh("results/" +"room_mesh_test.msh", MPI.COMM_WORLD)

# Access the mesh entities
cells = mesh.cells
facets = mesh.facets

# Add physical groups to the mesh entities
mesh.topology.create_connectivity(cells.dim, facets.dim)
gmsh.add_physical_group(mesh, cells.dim, [i for i in range(cells.shape[0])], label=1)  # Volume physical group
gmsh.add_physical_group(mesh, facets.dim, [i for i in range(facets.shape[0])], label=1)  # Surface physical group

# Define a volume from the surface entities
gmsh.add_volume(mesh, [1], tag=1)

# Save the modified mesh
gmsh.write_mesh("path_to_output_mesh.msh", mesh, cell_tags, facet_tags)


AttributeError: module 'dolfinx.io.gmshio' has no attribute 'read_msh'

In [56]:
# Choose if Gmsh output is verbose
gmsh.option.setNumber("General.Terminal", 0)

model = gmsh.model()
model.add("room")
model.setCurrent("room")
stl_path = os.path.join("results", "box.step")
room = gmsh.model.occ.importShapes(stl_path, highestDimOnly=False)



# Synchronize OpenCascade representation with gmsh model
model.occ.synchronize()



# Add physical marker for cells. It is important to call this function
# after OpenCascade synchronization

# Generate the mesh
model.mesh.generate(dim=3)
# model.add_physical_group(dim=3, tags=room)
model.addPhysicalGroup(3, [1])

# Create a DOLFINx mesh (same mesh on each rank)
msh, cell_markers, facet_markers = gmshio.model_to_mesh(model, MPI.COMM_SELF, 0)
msh.name = "room"
cell_markers.name = f"{msh.name}_cells"
facet_markers.name = f"{msh.name}_facets"






In [17]:
characteristic_length = 0.0017  # Desired mesh size
gmsh.model.mesh.setSize(gmsh.model.getEntities(3), characteristic_length)

gmsh.model.addPhysicalGroup(1, [line])

# Generate the mesh
gmsh.model.mesh.generate()

gmsh.write(os.path.join("results","cq_box.msh"))

# Finalize Gmsh
gmsh.finalize()


Info    : Meshing 1D...
Info    : Done meshing 1D (Wall 6.434e-06s, CPU 1e-05s)
Info    : Meshing 2D...
Info    : Done meshing 2D (Wall 4.669e-06s, CPU 3e-06s)
Info    : Meshing 3D...
Info    : Done meshing 3D (Wall 1.1898e-05s, CPU 7e-06s)
Info    : 0 nodes 0 elements
Info    : Writing 'results/cq_box.msh'...
Info    : Done writing 'results/cq_box.msh'
